# Анализ базы данных для создания приложения для продажи книг

# Описание проекта

Цель — проанализировать базу данных последством SQL-запросов.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.
Задачи:
- ***1 задача:*** Подсчёт книг вышедших после 1 января 2000 года;
- ***2 задача:*** Для каждой книги посчёт количество обзоров и среднюю оценку;
- ***3 задача:*** Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- ***4 задача:*** Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- ***5 задача:*** Посчёт среднего количество обзоров от пользователей, которые поставили больше 50 оценок.

# Описание данных для проекта

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Импорт библиотек, установка подключения к базе данных

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Вывод первых строк

In [4]:
query = '''SELECT *
           FROM books
           LIMIT 5;'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT *
           FROM authors
           LIMIT 5;'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT *
           FROM publishers
           LIMIT 5;'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT *
           FROM ratings
           LIMIT 5;'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''SELECT *
           FROM reviews
           LIMIT 5;'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Решение задач

### 1 задача

In [9]:
query = '''SELECT COUNT(book_id)
           FROM books
           WHERE publication_date > '2000-01-01';'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


Всего после 1 января 2000 года было опубликовано 819 книг

### 2 задача

In [10]:
query = '''SELECT b.book_id, b.title, a.count_review, a.avg_rating
           FROM (SELECT re.book_id, COUNT(DISTINCT re.review_id) count_review, AVG(rt.rating) avg_rating
           FROM ratings rt
           LEFT JOIN reviews re ON rt.book_id=re.book_id
           GROUP BY re.book_id) a
           LEFT JOIN books b ON a.book_id=b.book_id
           ORDER BY a.count_review DESC
           '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,948.0,Twilight (Twilight #1),7,3.662500
1,854.0,The Road,6,3.772727
2,656.0,The Book Thief,6,4.264151
3,734.0,The Glass Castle,6,4.206897
4,963.0,Water for Elephants,6,3.977273
...,...,...,...,...
990,465.0,Naked Empire (Sword of Truth #8),1,3.500000
991,446.0,Moo Baa La La La!,1,3.000000
992,431.0,Merrick (The Vampire Chronicles #7),1,4.000000
993,196.0,Dragons of Winter Night (Dragonlance: Chronicl...,1,4.000000


In [11]:
query = '''SELECT b.book_id, b.title, a.count_review, a.avg_rating
           FROM (SELECT bk.book_id, COUNT(DISTINCT re.review_id) count_review, ROUND(AVG(rt.rating), 2) avg_rating
           FROM books bk
           FULL JOIN ratings rt ON bk.book_id=rt.book_id
           FULL JOIN reviews re ON bk.book_id=re.book_id
           GROUP BY bk.book_id) a
           LEFT JOIN books b ON a.book_id=b.book_id
           ORDER BY a.count_review DESC
           '''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Максимальное количество обзоров на книгу под названием  Twilight (Twilight #1) и	составило 7, минимальное 0, Средний рейтинг рейтинг находится в диапозоне от 1.5 до 5.0

### 3 задача

In [10]:
query = '''SELECT pub.publisher, COUNT(b.book_id) count_books
           FROM books b
           LEFT JOIN publishers pub ON b.publisher_id = pub.publisher_id
           WHERE b.num_pages > 50
           GROUP BY pub.publisher
           ORDER BY count_books DESC
           LIMIT 1;'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


Издатель "Penguin Books" выпустил больше всех книг более 50 страниц, а именно 42 книги

### 4 задача

In [11]:
query = '''SELECT a.author, ROUND(AVG(rt.rating),2) avg_rating
           FROM authors a
           LEFT JOIN books b ON a.author_id = b.author_id
           LEFT JOIN ratings rt ON b.book_id = rt.book_id
           GROUP BY a.author
           HAVING COUNT(rt.rating_id) > 50
           ORDER BY avg_rating DESC
           LIMIT 1;'''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


"J.K. Rowling/Mary GrandPré" является автором с самой высокой средней оценкой(4.29 балла) среди авторов у которых больше 50 оценок среди написаных ими книг.

### 5 задача

In [24]:
query = '''SELECT ROUND(AVG(a.count_review), 2)
           FROM (SELECT rt.username, COUNT(DISTINCT re.review_id) count_review, COUNT(DISTINCT rt.rating_id) count_rating
           FROM ratings rt
           LEFT JOIN reviews re ON rt.username=re.username
           GROUP BY rt.username
           HAVING COUNT(DISTINCT rt.rating_id) > 50
           ORDER BY count_review) a;'''
pd.io.sql.read_sql(query, con = engine)

,round
0,24.33


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составило 164 обзора

## Общие выводы

- Всего после 1 января 2000 года было опубликовано 819 книг
- Максимальное количество обзоров на книгу  составлило 7, минимальное 0, Средний рейтинг рейтинг находится в диапозоне от 1.5 до 5.0
- Издатель "Penguin Books" выпустил больше всех книг более 50 страниц, а именно 42 книги
- "J.K. Rowling/Mary GrandPré" является автором с самой высокой средней оценкой(4.29 балла) среди авторов у которых больше 50 оценок среди написаных ими книг.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок составило 164 обзора